# Load libraries

In [1]:
import wandb
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
import tempfile
import os
import plotly.express as px

In [2]:
# Login to Weights & Biases
wandb.login(relogin=True)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Dell/.netrc


True

# ETL (Extract, Transform, Load)

In [3]:
from scipy import stats
from sklearn.preprocessing import MultiLabelBinarizer
from scipy.stats import pointbiserialr

In [4]:
# CREATING PROJETCT ON WANDB
"""
df= pd.read_csv("listings.csv.gz")
df['price'] = pd.to_numeric(df['price'].str.replace(",","").str.strip('$'))
df.to_csv("raw_data.csv",index=False)
!wandb artifact put \
      --name mlops-final-project/raw_data.csv \
      --type raw_data \
      --description "The raw data from of airbnb" raw_data.csv
"""

'\ndf= pd.read_csv("listings.csv.gz")\ndf[\'price\'] = pd.to_numeric(df[\'price\'].str.replace(",","").str.strip(\'$\'))\ndf.to_csv("raw_data.csv",index=False)\n!wandb artifact put       --name mlops-final-project/raw_data.csv       --type raw_data       --description "The raw data from of airbnb" raw_data.csv\n'

In [5]:
# save_code tracking all changes of the notebook and sync with Wandb
run = wandb.init(project="mlops-final-project", save_code=True)
# donwload the latest version of artifact raw_data.csv
artifact = run.use_artifact("mlops-final-project/raw_data.csv:latest")
# create a dataframe from the artifact
df = pd.read_csv(artifact.file())


wandb: Currently logged in as: victorvieirat (mlops-victor). Use `wandb login --relogin` to force relogin


In [6]:
df.head(2)

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,17878,https://www.airbnb.com/rooms/17878,20211224070558,2021-12-25,"Very Nice 2Br in Copacabana w. balcony, fast WiFi",Discounts for long term stays. <br />- Large b...,This is the one of the bests spots in Rio. Bec...,https://a0.muscache.com/pictures/65320518/3069...,68997,https://www.airbnb.com/users/show/68997,...,4.9,4.76,4.66,NaN,f,1,1,0,0,1.92
1,24480,https://www.airbnb.com/rooms/24480,20211224070558,2021-12-25,"Nice and cozy near Ipanema Beach, w/ home office","My studio is located in the best of Ipanema, t...","The beach, the lagoon, Ipanema is a great loca...",https://a0.muscache.com/pictures/11955612/b28e...,99249,https://www.airbnb.com/users/show/99249,...,4.9,4.97,4.58,NaN,f,1,1,0,0,0.62


## Columns
### Removing by Cardinality

In [7]:
cardCols = []
for col in df.columns:
    num = df[col].nunique()/df.shape[0]
    if num > 0.1:
        cardCols.append(col)
df[cardCols].head(2)

,id,listing_url,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_about,host_thumbnail_url,host_picture_url,latitude,longitude,amenities,first_review
0,17878,https://www.airbnb.com/rooms/17878,"Very Nice 2Br in Copacabana w. balcony, fast WiFi",Discounts for long term stays. <br />- Large b...,This is the one of the bests spots in Rio. Bec...,https://a0.muscache.com/pictures/65320518/3069...,68997,https://www.airbnb.com/users/show/68997,Matthias,2010-01-08,I am a journalist/writer. Lived in NYC for ...,https://a0.muscache.com/im/pictures/user/67b13...,https://a0.muscache.com/im/pictures/user/67b13...,-22.96599,-43.17940,"[""Bed linens"", ""Stove"", ""Elevator"", ""Cable TV""...",2010-07-15
1,24480,https://www.airbnb.com/rooms/24480,"Nice and cozy near Ipanema Beach, w/ home office","My studio is located in the best of Ipanema, t...","The beach, the lagoon, Ipanema is a great loca...",https://a0.muscache.com/pictures/11955612/b28e...,99249,https://www.airbnb.com/users/show/99249,Goya,2010-03-26,Welcome to Rio!\nI am a human being fighting f...,https://a0.muscache.com/im/pictures/user/6b404...,https://a0.muscache.com/im/pictures/user/6b404...,-22.98405,-43.20189,"[""Extra pillows and blankets"", ""Dining table"",...",2010-09-26


In [8]:
## Dessas colunas as únicas de nosso interesse são  host_since,latitude,longitude,first_review
[cardCols.remove(x) for x in ['host_since','latitude','longitude','first_review','amenities']]
df = df.drop(cardCols, axis=1)
df.head(2)

,scrape_id,last_scraped,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,20211224070558,2021-12-25,2010-01-08,"Rio de Janeiro, State of Rio de Janeiro, Brazil",within an hour,100%,96%,f,Copacabana,2.0,...,4.9,4.76,4.66,NaN,f,1,1,0,0,1.92
1,20211224070558,2021-12-25,2010-03-26,"Rio de Janeiro, Rio de Janeiro, Brazil",a few days or more,100%,100%,f,Ipanema,0.0,...,4.9,4.97,4.58,NaN,f,1,1,0,0,0.62


### Removing by missing values

In [9]:
missing = (df.isna().sum()/len(df)).sort_values(ascending=False)
missing[:20]

license                         1.000000
neighbourhood_group_cleansed    1.000000
calendar_updated                1.000000
bathrooms                       1.000000
host_neighbourhood              0.479897
neighbourhood                   0.461811
review_scores_location          0.380341
review_scores_accuracy          0.380301
review_scores_cleanliness       0.380301
review_scores_checkin           0.380301
review_scores_communication     0.380260
review_scores_value             0.380260
first_review                    0.365025
last_review                     0.365025
review_scores_rating            0.365025
reviews_per_month               0.365025
host_acceptance_rate            0.230030
host_response_time              0.158092
host_response_rate              0.158092
bedrooms                        0.060695
dtype: float64

**We will remove the columns with more than 35% of missing values**

In [10]:
df = df.drop(missing[missing > 0.35].index, axis=1)
df.head(2)

,scrape_id,last_scraped,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,...,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms
0,20211224070558,2021-12-25,2010-01-08,"Rio de Janeiro, State of Rio de Janeiro, Brazil",within an hour,100%,96%,f,2.0,2.0,...,257,2021-12-25,267,10,1,f,1,1,0,0
1,20211224070558,2021-12-25,2010-03-26,"Rio de Janeiro, Rio de Janeiro, Brazil",a few days or more,100%,100%,f,0.0,0.0,...,107,2021-12-25,85,0,0,f,1,1,0,0


**Removing other useless columns**

In [11]:
#Drop Useless
df = df.drop(['scrape_id','last_scraped','host_location','calendar_last_scraped','host_verifications','property_type'], axis=1)

### Rows

In [12]:
missing = (df.isna().sum()/len(df)).sort_values(ascending=False)
missing[missing.values > 0]

host_acceptance_rate         0.230030
host_response_rate           0.158092
host_response_time           0.158092
bedrooms                     0.060695
beds                         0.014135
bathrooms_text               0.002607
host_has_profile_pic         0.000611
host_since                   0.000611
host_total_listings_count    0.000611
host_listings_count          0.000611
host_is_superhost            0.000611
host_identity_verified       0.000611
dtype: float64

In [13]:
#%%capture
# Changin host since to month count
df['host_since'] =  pd.to_datetime(df['host_since'], format='%Y-%m-%d')
def diff_month(x):
    curr_time = pd.to_datetime("now")
    return (x.year - curr_time.year) * 12 + x.month - curr_time.month
df['host_since'] = df['host_since'].apply(lambda x: diff_month(x))

In [14]:
#Changing bathrooms text to numeric
df.loc[df['bathrooms_text'].str.contains('half', na=False), 'bathrooms_text'] = '0.5'
df['bathrooms_text'] = pd.to_numeric(df['bathrooms_text'].str.replace("[a-zA-Z\s-]",""))

<ipython-input-14-98ab8868ab8f>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['bathrooms_text'] = pd.to_numeric(df['bathrooms_text'].str.replace("[a-zA-Z\s-]",""))


In [15]:
## Removing Percentege
for colName in ['host_response_rate','host_acceptance_rate']:
    df[colName ] = df[colName ].str.rstrip('%').astype('float')/100.0

In [16]:
## Changin Categorical NA to mode
for colName in ['host_has_profile_pic','host_is_superhost','host_identity_verified','host_response_time','host_since']:
    df[colName ] = df[colName].fillna(df[colName].mode()[0])
for colName in ['host_acceptance_rate','host_response_rate','bedrooms','beds',
                'bathrooms_text','host_total_listings_count','host_listings_count']:
    df[colName ] = df[colName].fillna(round(df[colName].mean(),2))


In [17]:
missing = (df.isna().sum()/len(df)).sort_values(ascending=False)
missing[missing.values > 0]

Series([], dtype: float64)

## Converting Categorical Ordinal/Binary to  numeric

In [18]:
df['host_response_time']= df['host_response_time'].replace(['within an hour','within a few hours', 'within a day', 'a few days or more'],
                        [1, 2,3,4])
df['room_type']= df['room_type'].replace(['Shared room', 'Private room',  'Hotel room','Entire home/apt'],
                        [1, 2,3,4])
for colName in ['host_is_superhost','host_has_profile_pic','host_identity_verified','has_availability','instant_bookable']:
    df[colName] = df[colName].replace(['f','t'],
                            [0,1])

## Binary Data
We will check the impact of binary data on price using  [biserial correlation coefficient and its p-value](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mstats.pointbiserialr.html). When p is less than 0.5, we will use it on our project

### Neighbourhood
we could use Latitude  and Longitude instead of the Neighbourhood name but given the lack of time we had we choose this aprroach

In [19]:
district= pd.get_dummies(df['neighbourhood_cleansed'])
if 'dfdistrict' in locals() : del dfdistrict
y = df['price']
for colName in district.columns:
    #calculate point-biserial correlation
    x = district[colName]
    corr, pVal = pointbiserialr(x, y)
    dfTemp = {'column': colName, 
            'correlation': corr, 
            'pVal': pVal}
    if 'dfdistrict' in locals():
        dfdistrict = dfdistrict.append(dfTemp, ignore_index = True)
    else:
        dfdistrict=pd.DataFrame(dfTemp, index=[0])
importantDistrict = dfdistrict.loc[dfdistrict['pVal'] < 0.5].sort_values(by='correlation', ascending=False)['column']
districs = district[importantDistrict]
districs.head(2)

,São Cristóvão,Joá,São Conrado,Ipanema,Leblon,Santa Cruz,Barra da Tijuca,Alto da Boa Vista,Lagoa,Itanhangá,...,Laranjeiras,Camorim,Leme,Catete,Santa Teresa,Jacarepaguá,Tijuca,Flamengo,Centro,Botafogo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Amenities
Amenities is a variable with a list of things that have in the room. 

In [20]:
listValues = df['amenities'].apply(lambda x: x.strip('][').split(', '))
def myfunc(a):
  return a.replace('"','')
df['amenities'] = listValues.apply(lambda x: list(map(myfunc, x)))

mlb = MultiLabelBinarizer(sparse_output=True)

dfAmenities = df['price'].to_frame().join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(df.pop('amenities')),
                index=df.index,
                columns=mlb.classes_))

In [21]:
if 'dfBi' in locals() : del dfBi 
    
y = dfAmenities['price']
for colName in dfAmenities.columns[1:]:
    #calculate point-biserial correlation
    x = dfAmenities[colName]
    corr, pVal = pointbiserialr(x, y)
    dfTemp = {'column': colName, 
            'correlation': corr, 
            'pVal': pVal}
    if 'dfBi' in locals():
        dfBi = dfBi.append(dfTemp, ignore_index = True)
    else:
        dfBi=pd.DataFrame(dfTemp, index=[0])
        
dfBi = dfBi.loc[dfBi['pVal'] < 0.5].sort_values(by='correlation', ascending=False)
importantCols = dfBi['column']
dfAmenities = dfAmenities[dfBi['column'].values]

In [22]:
dfAmenities.head(2)

,Hot tub,Pool,Free parking on premises,Air conditioning,Beachfront,Dishwasher,Bathtub,Dryer,Crib,TV with standard cable,...,Cleaning products,Host greets you,Extra pillows and blankets,Luggage dropoff allowed,Ceiling fan,Cooking basics,Dishes and silverware,Essentials,Hot water,Lock on bedroom door
0,1,0,0,1,0,0,0,0,0,1,...,0,0,0,1,0,1,1,1,1,0
1,0,0,0,1,0,0,0,0,0,1,...,1,0,1,0,1,0,1,1,1,0


### Other Binary Data

In [23]:
binary= df[['host_is_superhost','host_has_profile_pic','host_identity_verified','has_availability','instant_bookable']]
y = df['price']
if 'dfbinary' in locals() : del dfbinary 
for colName in binary.columns:
    #calculate point-biserial correlation
    x = binary[colName]
    corr, pVal = pointbiserialr(x, y)
    dfTemp = {'column': colName, 
            'correlation': corr, 
            'pVal': pVal}
    if 'dfbinary' in locals():
        dfbinary = dfbinary.append(dfTemp, ignore_index = True)
    else:
        dfbinary=pd.DataFrame(dfTemp, index=[0])
importantbinary = dfbinary.loc[dfbinary['pVal'] < 0.5].sort_values(by='correlation', ascending=False)['column']
binary = binary[importantbinary]
binary

,has_availability
0,1
1,1
2,1
3,1
4,1
...,...
24544,1
24545,1
24546,1
24547,1


In [24]:
binary = binary.join(dfAmenities).join(districs)
binary

,has_availability,Hot tub,Pool,Free parking on premises,Air conditioning,Beachfront,Dishwasher,Bathtub,Dryer,Crib,...,Laranjeiras,Camorim,Leme,Catete,Santa Teresa,Jacarepaguá,Tijuca,Flamengo,Centro,Botafogo
0,1,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24544,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24545,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
24546,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
24547,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Ordinal and Numeric
now let's see which collumns use on our project using pandas profiling

In [25]:
notOrdinal = ['host_is_superhost','host_has_profile_pic','host_identity_verified','has_availability','instant_bookable',
              'neighbourhood_cleansed','latitude','longitude']
df = df.drop(columns=notOrdinal)
df

,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_listings_count,host_total_listings_count,room_type,accommodates,bathrooms_text,bedrooms,...,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms
0,-150.0,1,1.00,0.96,2.0,2.0,4,5,1.0,2.0,...,22,24,257,267,10,1,1,1,0,0
1,-148.0,4,1.00,1.00,0.0,0.0,4,2,1.0,1.0,...,0,0,107,85,0,0,1,1,0,0
2,-147.0,3,0.80,0.41,3.0,3.0,4,3,1.0,1.0,...,10,26,206,238,0,0,1,1,0,0
3,-145.0,1,1.00,0.72,1.0,1.0,4,2,1.5,1.0,...,0,0,207,181,0,0,1,1,0,0
4,-145.0,1,1.00,0.97,1.0,1.0,4,2,1.5,1.0,...,3,8,58,378,32,2,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24544,-50.0,1,0.79,0.72,0.0,0.0,4,10,3.0,4.0,...,60,90,365,0,0,0,1,1,0,0
24545,-95.0,1,1.00,1.00,1.0,1.0,4,4,1.0,1.0,...,3,3,3,1,1,1,4,4,0,0
24546,-74.0,1,0.79,0.72,1.0,1.0,4,8,2.0,3.0,...,57,87,267,0,0,0,1,1,0,0
24547,-86.0,2,1.00,0.98,9.0,9.0,4,6,1.0,1.0,...,51,81,81,0,0,0,4,4,0,0


In [26]:
#ProfileReport(df, title="Pandas Profiling Report").to_file("report.html")

**Lets remove only the higly correlated values acording to pandas profiligin**

In [27]:
df = df.drop(['maximum_nights_avg_ntm','maximum_maximum_nights','minimum_maximum_nights','maximum_minimum_nights',
        'minimum_minimum_nights','maximum_nights','minimum_nights','availability_30','availability_60',
         'calculated_host_listings_count_private_rooms'], axis=1)

**Acording to pandas profiling the price ang minimum_nights_avg_ntm is highly skewed 76 and 22**

Is really important to reduce this because some models assume:

- Linearity: assumes that the relationship between predictors and target variable is linear

- No noise: eg. that there are no outliers in the data

- No collinearity: if you have highly correlated predictors, it’s most likely your model will overfit

- Normal distribution: more reliable predictions are made if the predictors and the target variable are normally distributed

- Scale: it’s a distance-based algorithm, so preditors should be scaled — like with standard scaler

In [28]:
df = df.join(binary)

In [29]:
df

,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_listings_count,host_total_listings_count,room_type,accommodates,bathrooms_text,bedrooms,...,Laranjeiras,Camorim,Leme,Catete,Santa Teresa,Jacarepaguá,Tijuca,Flamengo,Centro,Botafogo
0,-150.0,1,1.00,0.96,2.0,2.0,4,5,1.0,2.0,...,0,0,0,0,0,0,0,0,0,0
1,-148.0,4,1.00,1.00,0.0,0.0,4,2,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
2,-147.0,3,0.80,0.41,3.0,3.0,4,3,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3,-145.0,1,1.00,0.72,1.0,1.0,4,2,1.5,1.0,...,0,0,0,0,0,0,0,0,0,0
4,-145.0,1,1.00,0.97,1.0,1.0,4,2,1.5,1.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24544,-50.0,1,0.79,0.72,0.0,0.0,4,10,3.0,4.0,...,0,0,0,0,0,0,0,0,0,0
24545,-95.0,1,1.00,1.00,1.0,1.0,4,4,1.0,1.0,...,0,0,0,0,0,0,0,1,0,0
24546,-74.0,1,0.79,0.72,1.0,1.0,4,8,2.0,3.0,...,0,0,1,0,0,0,0,0,0,0
24547,-86.0,2,1.00,0.98,9.0,9.0,4,6,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
# Save the artifacts. We use a temporary directory so we do not leave
# any trace behind

with tempfile.TemporaryDirectory() as tmp_dir:
        # Make the artifact name from the provided root plus the name of the split
        artifact_name = "preprocessed_data.csv"
        # Get the path on disk within the temp directory
        temp_path = os.path.join(tmp_dir, artifact_name)
        # Save then upload to W&B
        df.to_csv(temp_path,index=False)
        artifact = wandb.Artifact(
            name=artifact_name,
            type="raw_data",
            description=f"preprocessed data of dataset mlops-final-project/raw_data.csv:latest",
        )
        artifact.add_file(temp_path)
        run.log_artifact(artifact)
        # This waits for the artifact to be uploaded to W&B. If you
        # do not add this, the temp directory might be removed before
        # W&B had a chance to upload the datasets, and the upload
        # might fail
        artifact.wait()

In [31]:
run.finish()